# Great Weapon Fighting
These are some math examples of the values you get for the Great Weapon Fighting style in 5e D&D
which let you reroll 1 or 2 on any weapon damage dice.


In [12]:
import numpy as np
import pandas as pd

# Pulling this model from AceCalhoon's answer
# https://rpg.stackexchange.com/questions/47172/how-much-damage-does-great-weapon-fighting-add-on-average

series = {}
for i in range(4, 13, 2):
    series[i] = 1 - (2/i)
df = pd.DataFrame(series.items(), columns=['Die Type', 'Difference'])
df

,Die Type,Difference
0,4,0.500000
1,6,0.666667
2,8,0.750000
3,10,0.800000
4,12,0.833333
